In [1]:
import scipy.optimize as so
import numpy as np
import h5py as h5
import scipy.linalg as la
np.set_printoptions(linewidth=110)
%matplotlib inline

In [2]:
def UFid(U,V,Dim):
#    return np.sum(abs(U-V)**2)
    return abs(np.trace(np.matmul(U.conj().T,V))/Dim)**2

In [3]:
def QuantumFourier(Dim): 
    pi=np.arccos(-1.)
    x=np.array(range(Dim))
    return np.exp( 1j*2*pi*np.reshape( np.kron(x,x), [Dim,Dim] )/Dim )/np.sqrt(Dim)

In [4]:
def RandomUnitary(Dim):
    return np.linalg.qr( np.random.randn(Dim,Dim)+1j*np.random.randn(Dim,Dim) )[0]

In [5]:
def UFunction(Phase,N,F,Dim):
    U=np.eye(Dim)
    for k in range(N):
        U= np.exp( 1j*np.append([0] ,Phase[k*(Dim-1):(k+1)*(Dim-1)] ) )[:,np.newaxis]*U
        U=np.matmul(F,U)
    return np.exp(1j*np.append( [0], Phase[N*(Dim-1):(N+1)*Dim-1]))[:,np.newaxis]*U

In [12]:
def PUMIOpt(U,Dim,N,F,MC):
    Seed = np.ones( (N+1)*(Dim-1) )
    TargetOpt = lambda x: 1-UFid( U , UFunction( x , N , F , Dim ) , Dim )
    argsmin = {"method": "L-BFGS-B"}
    Funs = np.zeros(MC)
    for k in range(MC):
        Res=so.basinhopping(TargetOpt , Seed,minimizer_kwargs=argsmin,niter=100,niter_success=10,disp=False,T=1, stepsize=0.5*np.pi)
        Funs[k] = Res.fun
    return np.min(Funs)

In [7]:
# MC = 1000
# Dim = 3
# file = h5.File("Data_D"+str(Dim)+".h5","w")
# U = np.zeros([Dim,Dim,MC],dtype=complex)
# for k in range(MC):
#     U[:,:,k] = RandomUnitary(Dim)    
# file["re"] = np.real(U)
# file["im"] = np.imag(U)
# file["MC"] = [MC]
# file["Dim"] = [Dim]
# file.close()

In [7]:
def main(Name,Repe):

    file = h5.File(Name,"r")
    UU = file["re"][:,:,:]+1j*file["im"][:,:,:]
    MC = file["MC"][0]
    Dim = file["Dim"][0]
    file.close()
    
    N = Dim 
    F = QuantumFourier(Dim)
    Data = np.zeros(MC)
    
    for i in range(MC):
        Data[i] = PUMIOpt(UU[:,:,i], Dim, N, F, Repe)
    
    Indx1 = np.where(Data>10**(-9))
    Indx2 = np.where(Data<10**(-9))

    file = h5.File(Name,"w")
    file["Infidelity"] = Data
    file["Index1"] = Indx1[0]
    file["Index2"] = Indx2[0]
    file.close()
    

In [8]:
from scipy.io import loadmat
contraejemplos = loadmat('Datos_Matlab\Peores_Matrices.mat')
contraejemplos = contraejemplos['Peores_Matrices']

contraejemplos[0][0]

array([[ 0.23884392+0.97105797j,  0.        +0.j        ,  0.        +0.j        ],
       [ 0.        +0.j        , -0.04239556+0.70587667j,  0.50548216+0.49439714j],
       [ 0.        +0.j        , -0.27182898-0.65272487j,  0.671314  +0.22225383j]])

In [14]:
for k in range(8):
    Dim = k+3
    F = QuantumFourier(Dim)
    u = contraejemplos[0][k]
    a = PUMIOpt(u,Dim,Dim,F,10)
    b = PUMIOpt(u,Dim,Dim+1,F,10)
    print(Dim,a,b)

0.04969938835048471 1.0258460747536446e-13
0.10563868125212017 2.050315472956754e-11
0.0035309624729135125 4.208569048813615e-10
1.0595162980298056e-07 1.5423744503806347e-09
7.351038888714356e-08 2.590361258825169e-09
2.3591773101738767e-07 3.814607518748403e-09
1.396119597707468e-07 9.67771041082699e-09
4.814659903318308e-06 2.3736754006797867e-08


In [11]:
for k in range(7,8):
    Dim = k+3
    F = QuantumFourier(Dim)
    u = contraejemplos[0][k]
    a = PUMIOpt(u,Dim,Dim,F,20)
    #b = PUMIOpt(u,Dim,Dim+1,F,10)
    print(a)

3.9253158434782165e-06


In [ ]:
9.570259848934448e-06

In [13]:
for k in range(8):
    Dim = k+3
    F = QuantumFourier(Dim)
    u = contraejemplos[0][k]
    a = PUMIOpt(u,Dim,Dim,F,10)
    b = PUMIOpt(u,Dim,Dim+1,F,10)
    
    print(a,b)

0.049699388350453955 8.066880496926387e-13
0.10563868122752851 9.890421814873207e-11
0.003530962665704629 4.4538017718309675e-10
1.312836576117249e-07 1.5249734808264748e-09
5.153865112461631e-08 1.8159294068453846e-09
1.9067128764760355e-07 8.13974687652319e-09
8.44460623872223e-07 1.1704057500949716e-08
4.673313438563476e-06 1.6192011176308085e-08


In [47]:
file = h5.File("Data_D3.h5","r")
UU = file["re"][:,:,:]+1j*file["im"][:,:,:]
Indx1 = file["Index1"][:]
Indx2 = file["Index2"][:]
file.close()

KeyError: "Unable to open object (object 're' doesn't exist)"

In [51]:
len(Indx1)

In [35]:
len(Indx2)

848

In [76]:
Dim = 10
N = Dim
F = QuantumFourier(Dim)
u = la.block_diag(RandomUnitary(int(Dim/2)),RandomUnitary(int(Dim/2)))
#u = RandomUnitary(Dim)

In [16]:
print(u)

[[-0.13420407-0.67380011j  0.12810513-0.45898462j -0.29031841+0.22075578j  0.11982682-0.18992923j
   0.32304828+0.11443431j  0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j          0.        +0.j        ]
 [-0.04557613-0.38588869j -0.38743946+0.13390392j -0.33601173-0.61624735j -0.06505159+0.04962541j
  -0.16885881-0.39128095j  0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j          0.        +0.j        ]
 [ 0.34089718+0.35506785j -0.48034116-0.03907953j -0.40104313+0.08663653j  0.16443967-0.53342133j
   0.18567303+0.10519704j  0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j          0.        +0.j        ]
 [ 0.23322838+0.22129383j  0.12436423-0.58229147j -0.29829863+0.14641695j -0.46092854+0.16416027j
  -0.30840232-0.31171789j  0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j          0.        +0.j        ]
 [ 0.06760227-0.16363021j  0.11388075-0.0716

In [80]:
%%time
PUMIOpt(u,Dim,N,F,1)

Wall time: 36.9 s


3.0761140374813145e-05

In [14]:
int(Dim/2)

5

In [22]:
15*1000/60

250.0

In [17]:
from scipy.io import loadmat
contraejemplos = loadmat('contraejemplos.mat')
contraejemplos = contraejemplos['contraejemplos']

In [36]:

for l in range(2,10):
    for k in range(8):
        dim = 3+k
        U = contraejemplos[k,0]
        Fids[0,k,l]=PUMIOpt(U,dim,dim,QuantumFourier(dim),1)
        Fids[1,k,l]=PUMIOpt(U,dim,dim+1,QuantumFourier(dim),1)
   

In [47]:
for k in range(8):
    print(np.min(Fids[0,k,:]))

0.049560046670114155
0.04000620037681535
9.260274263944712e-05
4.2568404312604e-07
5.86615982789418e-07
7.220394748763681e-08
4.6669880293670474e-07
4.604928143470133e-06


In [48]:
for k in range(8):
    print(np.min(Fids[1,k,:]))

5.031530747601209e-13
6.069100777494896e-11
5.097331445824693e-10
8.573468601724699e-10
3.0491704716695267e-09
4.097321371077101e-09
9.249504051567214e-09
2.8156744935614597e-08


In [74]:
contraejemplos[3,2]

array([[3.36619621e-12]])

In [75]:
Fids[1,7,:]

array([4.35137579e-08, 2.81567449e-08, 3.46371291e-08, 2.96424518e-08, 3.93145786e-08, 5.30666251e-08,
       5.18803085e-08, 3.70629197e-08, 4.40645280e-08, 4.02838058e-08])

In [ ]:
PUMIOpt(U,dim,dim,QuantumFourier(dim),1)